In [43]:
import pandas as pd
import requests
import time
import json
from tqdm import tqdm

df = pd.read_csv('UnivCity.csv')

In [45]:
df['CITY'] = 'Philadelphia'
df['DISTRICT'] = 'University City'
df['REGION'] = 'West Philadelphia'

df['preciselyID'] = None
df['livingSquareFootage'] = None
df['saleAmount'] = None

access_token = 'Bearer eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJON1NUbF9iQVo2Qk93MXZGUkNYM1Q2OWsyR0laTGw1cVd2MmZyMTdvTk1BIn0.eyJleHAiOjE3Mjg0MDY4NzUsImlhdCI6MTcyODQwMzI3NSwianRpIjoiOTViYjhiNzQtMjQxZC00N2NjLWJjYzktOWY0NGYwYWM4YTNmIiwiaXNzIjoiaHR0cHM6Ly9hdXRoLmNsb3VkLnByZWNpc2VseS5jb20vYXV0aC9yZWFsbXMvUHJlY2lzZWx5IiwiYXVkIjoiUHJlY2lzZWx5Iiwic3ViIjoiYzZmMGYxYTYtNGUzNy00NGVjLWFhMTItZGU5M2RkYjNkYzk2IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoiT0lEQy1ESVMtU2hlbGwiLCJzZXNzaW9uX3N0YXRlIjoiNjNmOGNkZDUtZjVkYi00Yjk4LWE2MjEtMGJlMDkxNjUwYmMwIiwiYWxsb3dlZC1vcmlnaW5zIjpbIioiXSwic2NvcGUiOiJ1cG4gcHJvZmlsZSBlbWFpbCIsInNpZCI6IjYzZjhjZGQ1LWY1ZGItNGI5OC1hNjIxLTBiZTA5MTY1MGJjMCIsInJlc291cmNlX2FjY2VzcyI6eyJPSURDLUNvbm5lY3QiOnsicm9sZXMiOlsiRGVzaWduZXIiLCJPcGVyYXRvciJdfSwiT0lEQy1ESVMtQ3VzdG9tZXJBY2NvdW50TWFuYWdlbWVudCI6eyJyb2xlcyI6WyJhZG1pbiIsImFnZW50YWRtaW5pc3RyYXRvciJdfSwiT0lEQy1EaXNjb3ZlcnkiOnsicm9sZXMiOlsiYWRtaW4iLCJ1c2VyIl19LCJicm9rZXIiOnsicm9sZXMiOlsicmVhZC10b2tlbiJdfSwiT0lEQy1ESVMtRFFDb3JlIjp7InJvbGVzIjpbIkRlc2lnbmVyIiwiT3BlcmF0b3IiXX19LCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwiY3VzdG9tZXJfYWNjb3VudF9pZHMiOlsiY25qdGEyeHcwbnhkdjhiaCJdLCJzdXJyb2dhdGVVc2VySWQiOiIwMHVtcGM2ajkxQVRGaGIxZTR4NyIsInByZWZlcnJlZF91c2VybmFtZSI6InJpc2hpa2EubGFsbEBkcmV4ZWwuZWR1IiwiZ2l2ZW5fbmFtZSI6IlJpc2hpa2EiLCJzdWJzY3JpcHRpb25faWQiOiJudWxsIiwidXBuIjoicmlzaGlrYS5sYWxsQGRyZXhlbC5lZHUiLCJjdXJyZW50X3dvcmtzcGFjZV9pZCI6ImNuanRhMnh3MG54ZHY4YmgiLCJlbnRpdGxlbWVudF9pZCI6Im51bGwiLCJuYW1lIjoiUmlzaGlrYSBMYWxsIiwicmVnaW9uIjoiVVMxIiwiZmFtaWx5X25hbWUiOiJMYWxsIiwiZW1haWwiOiJyaXNoaWthLmxhbGxAZHJleGVsLmVkdSJ9.l2Mr6aZaQ89JXatD5Q2ejDDHielpF58MwP0B-cbHtwfKu7-JsJz5SfSOQv946cTjB_Y9GSMm6Jeo8KfQuZLWZU_VkkkTdTItHqPgrOokhO2_L5GF1OsL6DoaoteAAAOKcQymV796n5Eq8iAF2cWNBIvg5TMAaNWbOhe5zW2P8ha_8pwAJuQd_66dIcrGjnIKInS_fy1hrtU7jtVW6WSzSUiOwissfz1sIDuS4R_JADqTHj1r1HIP49VO8qIVuClFBZMzvgOSFkDIbGJDZjUZCp6JfBTvRrKsRx6tvHURanJP_UPqSyRtE1i6AJkB3MUwstJ8G2lrAplycxixPwDV3A'

headers = {
  'cache-disabled': 'true',
  'Content-Type': 'application/json',
  'Authorization': access_token
}

url = "https://api.cloud.precisely.com/data-graph/graphql"

def fetch_property_data(row):
    address = f"{row['NUMBER']} {row['STREET']}, {row['CITY']} PA".strip()
    payload = f"""
    {{
      "query": "query propertyAttributesByAddress {{\\n  getByAddress(address: \\"{address}\\") {{\\n    propertyAttributes {{\\n      data {{\\n        preciselyID\\n        livingSquareFootage\\n        saleAmount\\n      }}\\n    }}\\n  }}\\n}}",
      "variables": {{}}
    }}
    """
    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code == 200:
        try:
            result = response.json()
            data = result['data']['getByAddress']['propertyAttributes']['data'][0]
            return data['preciselyID'], data.get('livingSquareFootage'), data.get('saleAmount')
        except (KeyError, IndexError, TypeError):
            return None, None, None
    else:
        return None, None, None

def update_dataframe_with_api_data(df):
    batch_size = 200
    total_entries = len(df)
    with tqdm(total=total_entries, desc="Processing", unit="row") as pbar:
        for i in range(0, total_entries, batch_size):
            batch = df[i:i + batch_size]
            for idx, row in batch.iterrows():
                preciselyID, livingSquareFootage, saleAmount = fetch_property_data(row)
                df.at[idx, 'preciselyID'] = preciselyID
                df.at[idx, 'livingSquareFootage'] = livingSquareFootage
                df.at[idx, 'saleAmount'] = saleAmount
                pbar.update(1)
            time.sleep(1)
    
    return df

df = update_dataframe_with_api_data(df)

Processing: 100%|██████████| 6947/6947 [24:53<00:00,  4.65row/s] 


In [48]:
def fetch_building_data(row):
    address = f"{row['NUMBER']} {row['STREET']}, {row['CITY']} PA".strip()
    payload = f"""
    {{
      "query": "query getBuildingByAddress {{\\n  getByAddress(address: \\"{address}\\") {{\\n    buildings {{\\n      data {{\\n        buildingID\\n        buildingType {{\\n          description\\n        }}\\n        elevation\\n        maximumElevation\\n        minimumElevation\\n        buildingArea\\n      }}\\n    }}\\n  }}\\n}}",
      "variables": {{}}
    }}
    """
    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code == 200:
        try:
            result = response.json()
            data = result['data']['getByAddress']['buildings']['data'][0]
            return (data.get('buildingID'), 
                    data['buildingType'].get('description'), 
                    data.get('elevation'), 
                    data.get('maximumElevation'), 
                    data.get('minimumElevation'), 
                    data.get('buildingArea'))
        except (KeyError, IndexError, TypeError):
            return None, None, None, None, None, None
    else:
        return None, None, None, None, None, None

def update_dataframe_with_building_data(df):
    batch_size = 200
    total_entries = len(df)
    with tqdm(total=total_entries, desc="Processing Buildings", unit="row") as pbar:
        for i in range(0, total_entries, batch_size):
            batch = df[i:i + batch_size]
            for idx, row in batch.iterrows():
                (buildingID, buildingType, elevation, maximumElevation, 
                 minimumElevation, buildingArea) = fetch_building_data(row)
                df.at[idx, 'buildingID'] = buildingID
                df.at[idx, 'buildingType'] = buildingType
                df.at[idx, 'elevation'] = elevation
                df.at[idx, 'maximumElevation'] = maximumElevation
                df.at[idx, 'minimumElevation'] = minimumElevation
                df.at[idx, 'buildingArea'] = buildingArea
                pbar.update(1)
            time.sleep(1)
    
    return df

# Initialize new columns in the dataframe
df['buildingID'] = None
df['buildingType'] = None
df['elevation'] = None
df['maximumElevation'] = None
df['minimumElevation'] = None
df['buildingArea'] = None

# Update the dataframe with building data
df = update_dataframe_with_building_data(df)

Processing Buildings: 100%|██████████| 6947/6947 [26:25<00:00,  4.38row/s]


In [54]:
df1 = pd.read_csv('UnivCity_first5k.csv')
df2 = pd.read_csv('UnivCity_last7k.csv')

df = pd.concat([df1, df2], axis=0, ignore_index=True)
df.shape

(11947, 20)

In [58]:
df.head

<bound method NDFrame.head of              LON        LAT NUMBER         STREET UNIT          CITY  \
0     -75.206209  39.969039   4102       OGDEN ST   AB  Philadelphia   
1     -75.207125  39.956518   4215    CHESTNUT ST    A  Philadelphia   
2     -75.207125  39.956518   4215    CHESTNUT ST    B  Philadelphia   
3     -75.207125  39.956518   4215    CHESTNUT ST    C  Philadelphia   
4     -75.207125  39.956518   4215    CHESTNUT ST    D  Philadelphia   
...          ...        ...    ...            ...  ...           ...   
11942 -75.210915  39.974724   4226  W THOMPSON ST  NaN  Philadelphia   
11943 -75.207302  39.963791   4203     WALLACE ST  NaN  Philadelphia   
11944 -75.202472  39.969104    848       UNION ST  NaN  Philadelphia   
11945 -75.207398  39.963783   4207     WALLACE ST  NaN  Philadelphia   
11946 -75.202222  39.967805    810       UNION ST  NaN  Philadelphia   

              DISTRICT             REGION  POSTCODE         ID  \
0      University City  West Philadelph